# Requirements:

In [ ]:
!pip install numpy==1.26.4

After executing the cell above, go to "Runtime" -> "Restart Session" (once). Otherwise the new (and needed) numpy version will not be recognized by Colab.

In [ ]:
!wget https://raw.githubusercontent.com/fabratu/nd24/main/networkit-11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!wget https://raw.githubusercontent.com/fabratu/nd24/main/tiny_01.graph

In [ ]:
%pip install --force-reinstall ./networkit-11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
%pip install seaborn plotly ipycytoscape networkx tabulate osmnx folium

The two cells above need also only to be executed once before you start with the content. Enjoy!

# NetworKit: Dynamic Algorithms, OpenStreetMap data, and advanced visualization tools

In [ ]:
import networkit as nk
from networkit import vizbridges
from networkit.vizbridges import Dimension
from networkit.dynamics import GraphEventType

import osmnx as ox
import networkx as nx
import seaborn as sns
import folium

import ast

# Dynamic Algorithms
When updating the graph (adding/removing nodes/edges), dynamic algorithms can compute the result for the updated graph directly from the previous result, without the need to re-run the entire algorithm.

We use two dynamic algorithms to compute properties of a dynamic graph:  
- `DynDijkstra`: find a shortest path
- `DynApproxBetweenness`: approximates betweenness centrality

First, load a graph and run both algorithms on this initial graph:

In [ ]:
# load a graph
G = nk.readGraph('tiny_01.graph', nk.Format.METIS)

# dijkstra algorithm with source node 0
dijkstra = nk.distance.DynDijkstra(G, 0)
dijkstra.run()

betweenness = nk.centrality.DynApproxBetweenness(G)
betweenness.run()

nk.overview(G)

Next, we want to visualize the graph, color code the nodes based on their centrality score, and draw the shortest path from node `0` to node `6`.

In [ ]:
# compute the path from souce to target using the results of the dijkstra algorithm
def getPath(dijkstra: nk.distance.DynDijkstra, source: int, target: int):
    path = []
    prev = target
    while prev != source:
        path.append(prev)
        prev = dijkstra.getPredecessors(prev)[-1]
    path.append(prev)
    path.reverse()
    return path

# visualization function
def visualizeDistances(G: nk.Graph, dijkstra: nk.distance.DynDijkstra, centrality: nk.centrality.DynApproxBetweenness):
    # get path from 0 to 6
    path = getPath(dijkstra, 0, 6)

    # create scores dictionary to visualize the path
    # 0 = edge not on path
    # 1 = edge on path
    pathScores = {(u,v): 0 for u,v in G.iterEdges()}
    for u,v in zip(path, path[1:]):
        pathScores[u,v] = 1

    # centrality scores as colors
    centralityScores = centrality.scores()

    # create widget
    G.indexEdges(force=True)
    widget = nk.vizbridges.widgetFromGraph(G, 
                                           nodeScores=centralityScores, 
                                           nodePalette=sns.light_palette("seagreen"), 
                                           edgeScores=pathScores, 
                                           edgePalette=[(0.2,0.2,0.85),(0,0,0)], 
                                           dimension=Dimension.Three
                                        )

    # modify the widget - more information in the pyplot documentation
    widget.update_layout(scene=dict(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False))
    widget.update_traces(marker=dict(size=50), line=dict(width=20))

    return widget


widget = visualizeDistances(G, dijkstra, betweenness)
widget.show()

#### Inspecting the widget


We can inspect the widget data while modifying layout and traces (=nodes and edges) to understand what we are changing. This widget is from `pyplot` - check the docs or google to see which attributes exist.

In [ ]:
widget.data

In [ ]:
widget.layout

## Updating the graph

The `run()` method does not need to be called again in order to adapt the result.
Instead, dynamic algorithms provide update methods to compute the new result.

In order to maintain consistent results, both the graph and the dynamic algorithm need to be adjusted.

- The [graph](https://networkit.github.io/dev-docs/python_api/graph.html?highlight=graph#module-networkit.graph) needs to be changed by using the graph manipulation function equivalent to the desired result - like `G.addEdge(...)` or `G.removeEdge(...)`.
- The dynamic algorithm needs to be changed, using [graph events](https://networkit.github.io/dev-docs/python_api/dynamics.html?highlight=graphevent#networkit.dynamics.GraphEvent) and calling the functions `update(...)` or `updateBatch(...)`.

For this example, we will remove two edges: `(0,4)` and `(3,6)`

In [ ]:
G.removeEdge(0,4)
event = nk.dynamics.GraphEvent(GraphEventType.EDGE_REMOVAL, 0, 4, 1)
dijkstra.update(event)
betweenness.update(event)

G.removeEdge(3,6)
event = nk.dynamics.GraphEvent(GraphEventType.EDGE_REMOVAL, 3, 6, 1)
dijkstra.update(event)
betweenness.update(event)


widget = visualizeDistances(G, dijkstra, betweenness)
widget.show()

## Using a Proxy to update dynamic algorithms

Updating multiple dynamic algorithms each time results in a lot of code. We can write a small proxy class for `Graph` to call the `update` function of all registered dynamic algorithms:

In [ ]:
class GraphEventProxy:
    def __init__(self, graph: nk.Graph):
        self._graph = graph
        self.observers = []
    
    def registerObserver(self, algorithm: nk.dynbase.DynAlgorithm):
        self.observers.append(algorithm)
    
    def addEdge(self, u, v, w=1.0, **kwargs):
        self._graph.addEdge(u,v,w,**kwargs)
        event = nk.dynamics.GraphEvent(GraphEventType.EDGE_ADDITION, u, v, w)
        for obs in self.observers:
            obs.update(event)
    
    @property
    def graph(self):    # sometimes we need direct access to the graph object itself
        return self._graph
    
    def __getattr__(self, attr):    # if we want to call a function of the graph object, __getattr__ will forward the call
        return getattr(self._graph, attr)
    

G = GraphEventProxy(G)
G.registerObserver(dijkstra)
G.registerObserver(betweenness)

G.addEdge(2,5)  # both dynamic algorithms are updated here
G.addEdge(0,3)

widget = visualizeDistances(G.graph, dijkstra, betweenness)  # the visualization function uses the graph object and not the proxy object
widget.show()

# Integration with other libraries: Using OpenStreetMap data

## Loading OSM data

We use `osmnx` to load the street network for Berlin from OSM. The resulting graph is a `networkx.MultiDiGraph`.

In [ ]:
location = "Mitte, Berlin, Germany"
# download location with OSMnx
oxGraph = ox.graph_from_place(
    location,
    network_type="drive",
    custom_filter='["highway"]["highway"!~"footway|bridleway|path|service|track|steps"]',
)

oxGraph.remove_edges_from(nx.selfloop_edges(oxGraph))
print(f"OSMnx Berlin graph has {oxGraph.number_of_edges()} edges")

Before converting the graph to a NetworKit graph, we need to aggregate the data of multi edges into a single edge:

In [ ]:
nxBerlinGraph = nx.DiGraph()
for u, v in oxGraph.edges():
    if not nxBerlinGraph.has_edge(u, v):
        nxBerlinGraph.add_edge(u, v, 
                                  geometry=list(list(d['geometry'].coords) if 'geometry' in d else [] for d in oxGraph[u][v].values()), # geometry data is aggregated into a list
                                  osmid=oxGraph[u][v][0]['osmid']) # osm way id of the edge

# store the osm node id as node attribute to make sure it is available as networkit node attribute later on
for n in nxBerlinGraph.nodes():
    nxBerlinGraph.nodes[n]['id'] = n  

nkBerlin = nk.nxadapter.nx2nk(nxBerlinGraph, data=True, typeMap={'osmid':str, 'id': str})
nk.overview(nkBerlin)

`nkBerlin` now has the following attributes:
- nodes: `id`: OSM node id
- edges:
  - `geometry`: list of coordinate line strings
  - `osmid`: OSM way id

## Visualizing the graph on a leaflet map

In [ ]:
def mapGraph(graph, highlightEdges=None):
    map = folium.Map(location=[52.5218, 13.4107], zoom_start=14)
    geometry = graph.getEdgeAttribute('geometry', str)
    wayIds = graph.getEdgeAttribute('osmid', str)
    nodeIds = graph.getNodeAttribute('id', str)
    for edge in graph.iterEdges():
        # turn the geometry data into list of coordinates
        for linestring in ast.literal_eval(geometry[edge]):
            if len(linestring) == 0:
                continue
            
            wayid = ""
            sourceid = ""
            targetid = ""
            try:
                wayid = wayIds[edge]
                sourceid = nodeIds[edge[0]]
                targetid = nodeIds[edge[1]]
            except:
                pass
            
            coords = [(y,x) for x,y in linestring]
            if highlightEdges and edge in highlightEdges:
                color = "#FF0000"
                weight = 5
            else:
                color = "#0000FF"
                weight = 2
            folium.PolyLine(locations=coords, 
                            color=color, 
                            weight=weight, 
                            tooltip=f"wayids={wayid} \
                                source={sourceid} \
                                target={targetid}"
                        ).add_to(map)
            
    return map

In [ ]:
mapGraph(nkBerlin)

We choose two nodes from this graph to run the dijkstra algorithm and draw the path on our map:

In [ ]:
nodeIds = nkBerlin.getNodeAttribute('id', str)
idmap = {nodeIds[node] : node for node in nkBerlin.iterNodes()}

source = idmap['660936117']
target = idmap['29218293']

dijkstra = nk.distance.DynDijkstra(nkBerlin, source)
dijkstra.run()
path = getPath(dijkstra, source, target)
pathEdges = [(u,v) for u,v in zip(path, path[1:])]
mapGraph(nkBerlin, pathEdges)

Removing the river bridge from the graph and updating the dynamic dijkstra algorithm:

In [ ]:
u = idmap['26747882']
v = idmap['26861434']

nkBerlin.removeEdge(u,v)
nkBerlin.removeEdge(v,u)
events = [
    nk.dynamics.GraphEvent(GraphEventType.EDGE_REMOVAL, u, v, 1),
    nk.dynamics.GraphEvent(GraphEventType.EDGE_REMOVAL, v, u, 1)
]
dijkstra.updateBatch(events)

path = getPath(dijkstra, source, target)
pathEdges = [(u,v) for u,v in zip(path, path[1:])]
mapGraph(nkBerlin, pathEdges)